In [2]:
# !pip install xgboost
# !pip install catboost
# !pip install scikit-learn
# !pip install pandas
# !pip install numpy
# !pip install tensorflow-gpu
# !pip install keras

In [3]:
import numpy as np
import pandas as pd
import keras
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding, LSTM
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.models import Sequential
from keras.callbacks import TensorBoard
import tensorflow as tf
from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras import backend as K
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer, CountVectorizer

Using TensorFlow backend.


Попробуем сделать предсказание на основе LSTM/GRU нейросети.

Загрузим данные и посмотрим на них

In [4]:
data_train = pd.read_csv('./data/train_data.tsv', sep='\t')
data_test = pd.read_csv('./data/test_data.tsv', sep='\t')

Напишем метрики для вызова в keras

In [5]:
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [6]:
#train_df.head()
data_train[data_train["answer"] == True]

,Unnamed: 0,ru_name,eng_name,answer
48,48,"Некоммерческое партнерство ""Компьютерная сеть ...",Microlan,True
62,62,"Общество с ограниченной ответственностью ""Торг...","Limited Liadility Company ""Trade House Achinsk""",True
76,76,"Общество с ограниченной ответственностью ""Венд...","""Vending"" Ltd",True
164,164,"Общество с ограниченной ответственностью ""Бухт...","""The Bay of Captains"" Ltd",True
182,182,"ЗАО ""Пристень-cахар""","Open Joint Stock Company ""Pristen-sugar""",True
...,...,...,...,...
3983094,3983094,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ САНРО...,SANROSE-2,True
3983106,3983106,"Общество с ограниченной ответственностью ""Трио...",TrioExpert,True
3983119,3983119,"ООО ""СВ-Финанс""","""SV-Finance"" LLC",True
3983178,3983178,"Общество с ограниченной ответственностью ""Эксе...","""Excellent""",True


In [7]:
data_test.head()

,Unnamed: 0,ru_name,eng_name
0,0,"ООО ""Технология-СМ""",TRANSMOROZ GMBH
1,1,Общество с ограниченной ответственностью Научн...,"""OBS"" Limited Liability Company"
2,2,"Общество с ограниченной ответственностью ""Торг...",Synthesis Engineering LTD
3,3,"ООО ""ТД ""Грумант""","""RMP Forwarder"" Company Limited"
4,4,"ООО ""Отис""","""Russian blinis"" limited liability company"


In [8]:
np.random.seed(7)

Сделаем предобработку.
Объединим нужные поля. Затем удалим лишние символы.

In [9]:
data_train["data"] = data_train["ru_name"] +" " + data_train["eng_name"]
data_train["data"]
data_train["answer"] = data_train["answer"].astype(int)

In [10]:
data_test["data"] = data_test["ru_name"] + " " + data_test["eng_name"]

In [11]:
# data_train.groupby('data')['data']

In [12]:
data_train["data"].replace("[^А-Яа-яA-Za-z0-9 ]", "", regex=True, inplace=True)
data_train["data"] = data_train["data"].str.lower()
data_test["data"].replace("[^А-Яа-яA-Za-z0-9 ]", "", regex=True, inplace=True)
data_test["data"] = data_test["data"].str.lower()

In [13]:
cts = data_train["data"].str.split(expand=True).stack().value_counts()
cts

общество            1832109
ооо                 1671662
с                   1664658
ограниченной        1653624
ответственностью    1651675
                     ...   
сибстройцентр             1
autocarrus                1
резервто                  1
дезмедсан                 1
агробелтревел             1
Length: 544152, dtype: int64

In [14]:
rep_dict = {"общество с ограниченной ответственностью": "", " ": ""}
for name in cts.index[:80]:
    if len(name) > 2:
        rep_dict[name] = ""

In [15]:
data_train["data"].replace(rep_dict, regex=True, inplace=True)

In [16]:
data_test["data"].replace(rep_dict, regex=True, inplace=True)

In [17]:
data_train["data_len"] = data_train['data'].str.len()

In [18]:
data_train[data_train["answer"] == 1]

,Unnamed: 0,ru_name,eng_name,answer,data,data_len
48,48,"Некоммерческое партнерство ""Компьютерная сеть ...",Microlan,1,некоммерческоепартнерствокомпьютернаясетьмикро...,57
62,62,"Общество с ограниченной ответственностью ""Торг...","Limited Liadility Company ""Trade House Achinsk""",1,ачинскliadilityachinsk,22
76,76,"Общество с ограниченной ответственностью ""Венд...","""Vending"" Ltd",1,вендингvending,14
164,164,"Общество с ограниченной ответственностью ""Бухт...","""The Bay of Captains"" Ltd",1,бухтакапитановbayofcaptains,27
182,182,"ЗАО ""Пристень-cахар""","Open Joint Stock Company ""Pristen-sugar""",1,пристеньcахарopenpristensugar,29
...,...,...,...,...,...,...
3983094,3983094,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ САНРО...,SANROSE-2,1,санрозе2sanrose2,16
3983106,3983106,"Общество с ограниченной ответственностью ""Трио...",TrioExpert,1,триоэкспертtrioexpert,21
3983119,3983119,"ООО ""СВ-Финанс""","""SV-Finance"" LLC",1,свфинансsvfinance,17
3983178,3983178,"Общество с ограниченной ответственностью ""Эксе...","""Excellent""",1,экселентexcellent,17


In [19]:
tokenizer = Tokenizer(
    char_level=True,
    lower=True
)

In [20]:
tokenizer.fit_on_texts([" ".join(data_train["data"])])

In [21]:
data_train[data_train["answer"] == 1].shape[0]/data_train.shape[0]

0.04591179510559718

Уменьшаем количество неправильных данных в датасете, и делаем соотношение ответов 1:3.

In [22]:
cleared = data_train[data_train["answer"] == 1]

In [23]:
cleared = cleared.append( data_train[data_train["answer"] == 0].head(cleared.shape[0]*3))

In [24]:
cleared = cleared.sample(frac=1).reset_index(drop=True)

In [25]:
cleared.head(100)

,Unnamed: 0,ru_name,eng_name,answer,data,data_len
0,522353,"Общество с ограниченной ответственностью ""Ай п...","Crystal-DV, Limited Liability Company",0,айcrystaldv,11
1,1652884,"ОАО ""БИЗНЕСГИПРОДОРИНВЕСТ""","""BIZNESGIPRODORINVEST""",1,бизнесгипродоринвестbiznesgiprodorinvest,40
2,119353,"ООО ""Вип-мастер""","""Quest-treid"" Limited Liability Company",0,випмастерquesttreid,19
3,2484224,"Общество с ограниченной ответственностью ""Ради...","""Radiopharmaceutical Preparations""",1,радиофармацевтическиепрепаратыradiopharmaceuti...,61
4,454192,"ООО ""Логика С""",SISTEMA LTD,0,логикасsistema,14
...,...,...,...,...,...,...
95,387155,"Закрытое акционерное общество ""КМВ - тур""","FIRM ""TECHOSTROY""",0,кмвтурfirmtechostroy,20
96,165937,"Общество с ограниченной ответственностью ""Конц...","""LADOGA LAKE"" Company Limited",0,концептфинансladogalake,23
97,407405,"ООО ""Игровые технологии""","Общество с ограниченной ответственностью ""SL P...",0,игровыеslpartner,16
98,463168,"ООО ""СМП ""Арсенал Безопасности""","Arsenal of Safety, Limited liability company",1,смпарсеналбезопасностиarsenalofsafety,37


In [38]:
def get_model1(max_features = 100):
    maxlen = 80
    batch_size = 32

    model = Sequential()
    model.add(Embedding(max_features, 256))
    model.add(LSTM(256, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(1, activation='sigmoid'))

    # try using different optimizers and different optimizer configs
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[f1_m])
    return model

In [27]:
data_test["data"]

0                                технологиясмtransmorozgmbh
1         научнопроизводственноерадиационныйконтрольприб...
2                                      сптsynsisengineering
3                                        тдгрумантrmpwarder
4                                                отисblinis
                                ...                        
996047    федеральноегосударственноебюджетноеобразовател...
996048                                           ритмsinton
996049                                             трубоwts
996050                                    евростройmonograd
996051                                        новыемедиаnew
Name: data, Length: 996052, dtype: object

In [28]:
# split a training set and a test set
X_train, X_valid, y_train, y_valid = train_test_split(cleared["data"], cleared["answer"], test_size=0.15, random_state=42)

Заменим символы на токены

In [29]:
X_test_t = np.apply_along_axis(tokenizer.texts_to_sequences, 0, data_test["data"])

In [30]:
X_test_t = pad_sequences(X_test_t, maxlen=100)

In [31]:
print(X_test_t)
print(data_test["data"])

[[ 0  0  0 ... 25 40 39]
 [ 0  0  0 ... 10 40 17]
 [ 0  0  0 ... 13 16 30]
 ...
 [ 0  0  0 ... 52 14 17]
 [ 0  0  0 ... 15  7 32]
 [ 0  0  0 ... 16  6 52]]
0                                технологиясмtransmorozgmbh
1         научнопроизводственноерадиационныйконтрольприб...
2                                      сптsynsisengineering
3                                        тдгрумантrmpwarder
4                                                отисblinis
                                ...                        
996047    федеральноегосударственноебюджетноеобразовател...
996048                                           ритмsinton
996049                                             трубоwts
996050                                    евростройmonograd
996051                                        новыемедиаnew
Name: data, Length: 996052, dtype: object


In [32]:
X_train_t = np.apply_along_axis(tokenizer.texts_to_sequences, 0, X_train)

In [33]:
X_train_t = pad_sequences(X_train_t, maxlen=100)

In [34]:
X_train_t

array([[ 0,  0,  0, ..., 10, 13, 20],
       [ 0,  0,  0, ..., 40, 37, 14],
       [ 0,  0,  0, ..., 28,  7, 17],
       ...,
       [ 0,  0,  0, ..., 13, 16, 30],
       [ 0,  0,  0, ...,  7, 16, 14],
       [ 0,  0,  0, ..., 13, 16, 30]])

In [39]:
model1 = get_model1(len(tokenizer.word_index) + 1)

In [36]:
y_train

604422    0
444527    1
71572     0
498877    0
683667    0
         ..
259178    1
365838    0
131932    1
671155    1
121958    0
Name: answer, Length: 621778, dtype: int32

Начнём обучение модели:

In [40]:
history = model1.fit(X_train_t, y_train, validation_split=0.25, epochs=8)

c:\users\user\appdata\local\continuum\anaconda3\envs\pytorch\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 466333 samples, validate on 155445 samples
Epoch 1/8
466333/466333 [==============================] - 1135s 2ms/step - loss: 0.3725 - f1_m: 0.5808 - val_loss: 0.2565 - val_f1_m: 0.7686
Epoch 2/8
466333/466333 [==============================] - 1162s 2ms/step - loss: 0.2383 - f1_m: 0.7876 - val_loss: 0.2023 - val_f1_m: 0.8275
Epoch 3/8
466333/466333 [==============================] - 1125s 2ms/step - loss: 0.2030 - f1_m: 0.8281 - val_loss: 0.1806 - val_f1_m: 0.8527
Epoch 4/8
466333/466333 [==============================] - 1121s 2ms/step - loss: 0.1847 - f1_m: 0.8486 - val_loss: 0.1737 - val_f1_m: 0.8625
Epoch 5/8
 21632/466333 [>.............................] - ETA: 16:44 - loss: 0.1762 - f1_m: 0.8586

KeyboardInterrupt: 

In [41]:
model1.save("NLP_char_model_0.84.h5")

In [42]:
answers = model1.predict_classes(X_test_t)

In [43]:
answers

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [1]])

In [44]:
ans_pd = pd.DataFrame({"answer": answers.flatten().astype(bool)})
ans_pd

,answer
0,False
1,False
2,False
3,True
4,False
...,...
996047,False
996048,False
996049,False
996050,False


In [45]:
ans_pd.to_csv("answer_neural_3.tsv", sep="\t", index=False)

In [ ]:
class CustomModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, units, batch_size):
        super(Model, self).__init__()
        self.units = units
        self.batch_size = batch_size
 
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
 
        self.gru = tf.keras.layers.GRU(self.units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_activation='sigmoid',
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)
 
    def call(self, inputs, hidden):
        inputs = self.embedding(inputs)
        #print(inputs)
        output, states = self.gru(inputs, initial_state=hidden)
 
        output = tf.reshape(output, (-1, output.shape[2]))
 
        x = self.fc(output)
 
        return x, states

keras_model = CustomModel(vocab_size, embedding_dim, units, BATCH_SIZE)